**Based on Peter Ferguson's notebook comparing EUCLID and ComCam data (/sdf/data/rubin/user/pferguso/projects/euclid/notebooks/euclid_star_compare.ipynb).**

## 1. User Input

See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs for the latest values.

In [ ]:
repo="/repo/main"
collection="LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"
instrument = 'LSSTComCam'
skymap = 'lsst_cells_v1'


## 2. Imports

(Many of these imports can be deleted.)

In [ ]:
# General python packages
import pylab as plt
import gc
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
#import lsst.afw.display as afwDisplay

#import lsst.pex.config as pexConfig
#import lsst.pipe.base as pipeBase

#from lsst.verify import Measurement
#from astropy import units as u

#from lsst.pex.config import Field, ListField

#import datetime
#import logging

#from lsst.analysis.tools.interfaces import MetricMeasurementBundle

import lsst.geom as geom
#from lsst.afw.image import Image
#from lsst.afw.geom import makeSkyWcs

#import healpy as hp


## 3. Useful functions

In [ ]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [ ]:
zeropoint = 31.4 # AB zeropoint
def flux2mag(flux):
    return -2.5*np.log10(flux) + zeropoint

## 4.  Main code

In [ ]:
# Create two butlers, one including the skymap.
#  (Creating two should not be necessary, but...)
butler = dafButler.Butler(repo, collections=collection)
skybutler = dafButler.Butler(repo, collections=collection, skymap=skymap)

In [ ]:
# Define which of the nearly 1000 columns to download from the ComCam ObjectTable...

INCOLS = [
    'coord_ra',
    'coord_dec',
    'detect_isPrimary',   
]
bands="ugrizy"
for band in bands:
    INCOLS += [
        f'{band}_psfFlux',
        f'{band}_psfFluxErr',
        f'{band}_ap12Flux',
        f'{band}_ap12FluxErr',
        f'{band}_extendedness',
        f'{band}_psfFlux_flag'
    ]


In [ ]:
# Choose which tract to of ComCam coadd catalog data to download.
# Here, we want the tract that contains the HST spectrophotometric standard, C26202, but it need not be this tract...

# RA, DEC of C26202 in degrees (from `/home/d/dltucker/DATA/SynthMags/synthMagColorList.lsst_v1.9.calspec_20240603.added_info.csv`):
raDeg = 53.136845833333325
decDeg = -27.86349444444444

# Create a SpherePoint for this RA, DEC...
my_point = geom.SpherePoint(raDeg * geom.degrees,
                            decDeg * geom.degrees)

# Grab the skymap from the skymap Butler and find the tract and patch
#  for this skymap that overlaps this RA,DEC...
skymap = skybutler.get('skyMap')

tract = skymap.findTract(my_point)
tractId = tract.tract_id

patch = tract.findPatch(my_point)
patchId = patch.getSequentialIndex()

print("""tract id: %d, patch id: %d""" % (tractId, patchId))

In [ ]:
# Download the raw ComCam data for this tract (or tract+patch)

# For full tract, use the objectTable_tract table...
raw_comcam = butler.get('objectTable_tract', dataId={'skymap': 'lsst_cells_v1', 'tract': tractId}, 
                        collections=[collection],
                        parameters={"columns":INCOLS})

# For just the given patch in the given tract, use the objectTable...
#raw_comcam = butler.get('objectTable', 
#                        dataId={'skymap': 'lsst_cells_v1', 'tract': tractId, 'patch': patchId}, 
#                        collections=[collection],
#                        parameters={"columns":INCOLS})


In [ ]:
# Clean the catalog

sel  = (raw_comcam['detect_isPrimary'] == True)
sel &= (raw_comcam['r_psfFlux']/raw_comcam['r_psfFluxErr'] > 10)
for band in bands:
    sel &= (raw_comcam[f'{band}_psfFlux_flag'] == 0)

comcam = raw_comcam[sel]


In [ ]:
# Find just the (most likely) stars...

sel_comcam_stars = (comcam['g_extendedness'] < 0.5) & (comcam['r_extendedness'] < 0.5)
#RA,DEC = comcam['coord_ra'].median(), comcam['coord_dec'].median()
#pix = hp.ang2pix(64, RA, DEC, lonlat=True, nest=True)
#upix = np.unique(hp.ang2pix(64, comcam['coord_ra'], comcam['coord_dec'], lonlat=True, nest=True))
#print(f"NSIDE=64, Nest, HEALPix: {pix}")

print(f"Number of objects: {len(comcam)}")
print(f"Number of stars: {sel_comcam_stars.sum()}")

In [ ]:
# print out the catalog of ComCam stars to the screen...

comcam[sel_comcam_stars]

In [ ]:
#filename = "comcamstars_cdfs.csv"
#comcam[sel_comcam_stars].to_csv(filename,index=False)

In [ ]:
# Stop here for now...
raise StopExecution

## 5. Sandbox

In [1]:
#repo="/repo/main"
#collection="LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"
#instrument = 'LSSTComCam'
#skymap = 'lsst_cells_v1'

repo = '/repo/main'
collection = 'LSSTCam/runs/DRP/20250604_20250921/w_2025_39/DM-52645'
instrument = 'LSSTCam'
skymap = 'lsst_cells_v1'


In [2]:
# General python packages
import pylab as plt
import gc
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
#import lsst.afw.display as afwDisplay

#import lsst.pex.config as pexConfig
#import lsst.pipe.base as pipeBase

#from lsst.verify import Measurement
#from astropy import units as u

#from lsst.pex.config import Field, ListField

#import datetime
#import logging

#from lsst.analysis.tools.interfaces import MetricMeasurementBundle

import lsst.geom as geom
#from lsst.afw.image import Image
#from lsst.afw.geom import makeSkyWcs

#import healpy as hp


In [3]:
# Create two butlers, one including the skymap.
#  (Creating two should not be necessary, but...)
butler = dafButler.Butler(repo, collections=collection)
skybutler = dafButler.Butler(repo, collections=collection, skymap=skymap)

In [4]:
# Define which of the nearly 1000 columns to download from the ComCam ObjectTable...

INCOLS = [
    'coord_ra',
    'coord_dec',
    'detect_isPrimary',   
]
bands="ugrizy"
for band in bands:
    INCOLS += [
        f'{band}_psfFlux',
        f'{band}_psfFluxErr',
        f'{band}_ap12Flux',
        f'{band}_ap12FluxErr',
        f'{band}_extendedness',
        f'{band}_psfFlux_flag'
    ]


In [6]:
# Ask the registry for all objectTable_tract datasets in this collection
datasets = butler.registry.queryDatasets(
    datasetType="object",
    collections=collection,
    where="skymap='lsst_cells_v1'",
    findFirst=True
)

# Extract the tract IDs
tracts = sorted({ds.dataId["tract"] for ds in datasets})
print("Available tracts:", tracts)


Available tracts: [2078, 2079, 2080, 2233, 2234, 2235, 2236, 2237, 2392, 2393, 2394, 2395, 2396, 2397, 2534, 2535, 2536, 2537, 2558, 2559, 2560, 2561, 2562, 2701, 2702, 2703, 2704, 2705, 2706, 2729, 2730, 2731, 2874, 2875, 2876, 2877, 2878, 2879, 3052, 3053, 3054, 3055, 3056, 3233, 3234, 3235, 3236, 4636, 4637, 4638, 4847, 4848, 4849, 4850, 5061, 5062, 5063, 5064, 5065, 5211, 5212, 5213, 5214, 5215, 5216, 5217, 5218, 5219, 5220, 5221, 5222, 5223, 5224, 5225, 5226, 5279, 5280, 5281, 5282, 5429, 5430, 5431, 5432, 5433, 5434, 5435, 5436, 5437, 5438, 5439, 5440, 5441, 5442, 5443, 5444, 5445, 5446, 5651, 5652, 5653, 5654, 5655, 5656, 5657, 5658, 5659, 5660, 5661, 5662, 5663, 5664, 5665, 5666, 5667, 5668, 5875, 5876, 5877, 5878, 5879, 5880, 5881, 5882, 5883, 5884, 5885, 5886, 5887, 5888, 5889, 5890, 5891, 5892, 6101, 6102, 6103, 6104, 6105, 6106, 6107, 6108, 6109, 6110, 6111, 6112, 6113, 6114, 6115, 6116, 6117, 6118, 6330, 6331, 6332, 6333, 6334, 6335, 6336, 6337, 6338, 6339, 6340, 6341, 634